<a href="https://colab.research.google.com/github/RobDeutsche/CISC499/blob/main/Copy_of_Module15_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# Replace with your name
print("Rob Deutsche")

Rob Deutsche


# Build a Chatbot with Neural Network 

We've discovered how to build a chatbot with cosine similarity. Now, let's explore how we might build one with neural network!

We will create our training data, train a neural network with them, then use the trained model to make our chatbot.  

# 1. Install Libraries

Firstly, we will install libraries needed for this neural network powered chatbot. 
Keras is a machine learning library which utilizes tensorflow (another lower level machine learning library) at the backend. This makes it easier for us to deploy deep neural network for this purpose. 

In [1]:
# Task 1: Execute Library Imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense
 
from numpy import argmax
import numpy as np
import re

# 2. Input training data

We will first include the following training data for our chatbot:
1. X represent the different possible inputs that users might enter
2. Y represent the intent of the inputs

In [2]:
# Task 2: Create the possible input array
X = ['Hi',
     'Hello',
     'How are you?',
     'I am making',
     'making',
     'working',
     'studying',
     'see you later',
     'bye',
     'goodbye']

print(len(X))

10


In [3]:
# Task 3: Create the intent of the inputs
Y = ['greeting',
     'greeting',
     'greeting',
     'busy',
     'busy',
     'busy',
     'busy',
     'bye',
     'bye',
     'bye']

print(len(Y))

10


Notice that there are several different sentences that have similar intent. Here, we are only having 3 intents, but you can add as many as you want for your project!

This is the way our chatbot will work:
1. From the input sentence, we will identify the intent using our trained AI model.
2. For each intent, we have a prepared response. 

For example, if we identify that the intent of the input is for a greeting, we might ask the chatbot to reply with a greeting as well, something like 'hi' or 'how are you doing?'

We will use machine learning to create a model that can classify input sentence into different intents. 
We make it as follows:

1. We create a training data (X and Y above) which contains a list of sentences and their intents.
2. Use the training data to train a classifier. 
3. Vectorize input sentences and use classifier to determine intent. 

# 3. Text processing

As usual, we will start with text processing. Do you remember the process?

## 3.1 Remove non alphanumeric characters

In [4]:
# Task 4: Text preprocessing

def remove_non_alpha_numeric_characters(sentence):
    new_sentence = ''
    for alphabet in sentence:
        if alphabet.isalpha() or alphabet == ' ':
            new_sentence += alphabet
    return new_sentence

def preprocess_data(X):
    X = [data_point.lower() for data_point in X]
    X = [remove_non_alpha_numeric_characters(
        sentence) for sentence in X]
    X = [data_point.strip() for data_point in X]
    X = [re.sub(' +', ' ',
                data_point) for data_point in X]
    return X

# Create Vocabulary
X = preprocess_data(X)

vocabulary = set()
for data_point in X:
    for word in data_point.split(' '):
        vocabulary.add(word)

vocabulary = list(vocabulary)
print ("X :",X)
print ()
print ("Vocabulary :", vocabulary)

X : ['hi', 'hello', 'how are you', 'i am making', 'making', 'working', 'studying', 'see you later', 'bye', 'goodbye']

Vocabulary : ['am', 'later', 'see', 'hi', 'i', 'are', 'hello', 'you', 'studying', 'goodbye', 'how', 'working', 'making', 'bye']


## Create document vectors

In [5]:
# Task 5: Convert text data into array of numbers (bag of words) to be trained by neural network
X_encoded = []

def encode_sentence(sentence):
    sentence = preprocess_data([sentence])[0]
    sentence_encoded = [0] * len(vocabulary)
    for i in range(len(vocabulary)):
        if vocabulary[i] in sentence.split(' '):
            sentence_encoded[i] = 1
    return sentence_encoded

X_encoded = [encode_sentence(sentence) for sentence in X]
print (X_encoded)

[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0], [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]]


In [6]:
# Task 6: Convert intent text data into array of numbers (bag of words)
classes = list(set(Y))

Y_encoded = []
for data_point in Y:
    data_point_encoded = [0] * len(classes)
    for i in range(len(classes)):
        if classes[i] == data_point:
            data_point_encoded[i] = 1
    Y_encoded.append(data_point_encoded)
    
print (Y_encoded)

[[0, 0, 1], [0, 0, 1], [0, 0, 1], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0]]


# 4. Create training data and test data

In [7]:
# Task 7 Training & test data
X_train = X_encoded
y_train = Y_encoded
X_test = X_encoded
y_test = Y_encoded

print (y_test)

print(len(X_train))

y_train

[[0, 0, 1], [0, 0, 1], [0, 0, 1], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0]]
10


[[0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [0, 1, 0],
 [0, 1, 0]]

What does y_train represent? Do you understand the array shown above?

In [ ]:
#Y_train represents the classification of the intent for each response.  

# 5. Model training

Now we will use the training data to train our neural network.

In [8]:
# Task 9: Train ANN
model = Sequential()
model.add(Dense(units=64, activation='sigmoid',
                input_dim=len(X_train[0])))
model.add(Dense(units=len(y_train[0]), activation='softmax'))
model.compile(loss=categorical_crossentropy,
              optimizer=SGD(lr=0.01,
                            momentum=0.9, nesterov=True))
model.fit(np.array(X_train), np.array(y_train), epochs=500, batch_size=16)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/500
1/1 [==============================] - 1s 578ms/step - loss: 1.1180
Epoch 2/500
1/1 [==============================] - 0s 6ms/step - loss: 1.1118
Epoch 3/500
1/1 [==============================] - 0s 7ms/step - loss: 1.1041
Epoch 4/500
1/1 [==============================] - 0s 6ms/step - loss: 1.0962
Epoch 5/500
1/1 [==============================] - 0s 5ms/step - loss: 1.0889
Epoch 6/500
1/1 [==============================] - 0s 9ms/step - loss: 1.0831
Epoch 7/500
1/1 [==============================] - 0s 4ms/step - loss: 1.0790
Epoch 8/500
1/1 [==============================] - 0s 5ms/step - loss: 1.0765
Epoch 9/500
1/1 [==============================] - 0s 10ms/step - loss: 1.0752
Epoch 10/500
1/1 [==============================] - 0s 4ms/step - loss: 1.0747
Epoch 11/500
1/1 [==============================] - 0s 5ms/step - loss: 1.0743
Epoch 12/500
1/1 [==============================] - 0s 20ms/step - loss: 1.0737
Epoch 13/500
1/1 [==============================] - 0s 6m

## List down predictions

In [9]:
# Task 10: Calculate Predictions
predictions = [argmax(pred) for pred in model.predict(np.array(X_test))]

# Model Evaluation

Let's evaluate our model now. We will compare the prediction made by the model and our test data:

In [10]:
# Task 11: Evaluate Model
correct = 0
for i in range(len(predictions)):
    if predictions[i] == argmax(y_test[i]):
        correct += 1

print ("Correct:", correct)
print ("Total:", len(predictions))

Correct: 10
Total: 10


# Testing the chatbot

Let's test the chatbot now! We will input a sentence, and then see what class is predicted by the neural network:

In [12]:
# Task 12: Test
for i in range(3):
    print ("Enter a sentence")
    sentence = input()
    prediction= model.predict(np.array([encode_sentence(sentence)]))
    print (classes[argmax(prediction)])

Enter a sentence
My hovercraft is full of eels
busy
Enter a sentence
where am I?
busy
Enter a sentence
the cheese is no more
busy


# Challenge

We have successfully use neural network to map our input to conversation intent. 
Your challenge is to modify the code below so that it loops until the predicted class is 'bye'

In [14]:
# Task 13: Modify the code below so that it loops until the predicted class is 'bye'
while i in range(3):
    print ("Enter a sentence")
    sentence = input()
    prediction= model.predict(np.array([encode_sentence(sentence)]))
    #print (classes[argmax(prediction)])
    if (classes[argmax(prediction)] == 'bye'):
        print ('Goodbye! See you next time!')
        break
    elif (classes[argmax(prediction)] == 'busy'):
        print('ah... all the best!')
    elif (classes[argmax(prediction)] == 'greeting'):
        print ('Hi!')

Enter a sentence
hi
Hi!
Enter a sentence
how are you?
Hi!
Enter a sentence
Im very busy
ah... all the best!
Enter a sentence
bye
Goodbye! See you next time!


### Great job! You've successfully created a simple chatbot with neural network! How might you improve the chatbot?
You can improve the chatbot by:
- Adding more training data
- Adding more intent
- Focusing on a particular topic and train the chatbot with many training data in that topic

### Resource:
https://blog.eduonix.com/internet-of-things/simple-nlp-based-chatbot-python/